# ⚖️ Judge Subroutine Demo

Stress-test final verdict synthesis from prepared findings.

## 🔎 Sections
- ⚙️ Setup + auto-reload
- 🧱 Build synthetic findings
- 🚀 Run judge agent
- 🧪 Quick tweaks

## ⚙️ Setup + auto-reload

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import json
import os
import sys

ROOT = Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print(f"Root: {ROOT}")
print(f"Using src path: {SRC}")
print(f"OPENAI_API_KEY set: {bool(os.getenv('OPENAI_API_KEY'))}")

## 🧱 Build synthetic findings

In [ ]:
from facticli.types import AspectFinding, EvidenceSignal, InvestigationPlan, SourceEvidence, VerificationCheck

claim = "The Eiffel Tower was built in 1889 for the World's Fair."
# claim = "The first iPhone was released in 2008."
# claim = "Pluto is currently classified as a planet by the IAU."

plan = InvestigationPlan(
    claim=claim,
    checks=[
        VerificationCheck(
            aspect_id="timeline_1",
            question="Was Eiffel Tower completed in 1889?",
            rationale="Completion year is central.",
            search_queries=["Eiffel Tower completion year"],
        ),
        VerificationCheck(
            aspect_id="event_1",
            question="Was it associated with the 1889 World's Fair?",
            rationale="Event linkage is central.",
            search_queries=["Eiffel Tower 1889 Exposition Universelle"],
        ),
    ],
)

findings = [
    AspectFinding(
        aspect_id="timeline_1",
        question="Was Eiffel Tower completed in 1889?",
        signal=EvidenceSignal.SUPPORTS,
        summary="Multiple historical sources report completion in 1889.",
        confidence=0.9,
        sources=[
            SourceEvidence(
                title="Britannica - Eiffel Tower",
                url="https://www.britannica.com/topic/Eiffel-Tower-Paris-France",
                snippet="The Eiffel Tower was completed in 1889.",
            )
        ],
    ),
    AspectFinding(
        aspect_id="event_1",
        question="Was it associated with the 1889 World's Fair?",
        signal=EvidenceSignal.SUPPORTS,
        summary="Sources tie the tower to the Exposition Universelle held in Paris in 1889.",
        confidence=0.88,
        sources=[
            SourceEvidence(
                title="Paris tourism - Eiffel Tower history",
                url="https://www.toureiffel.paris/en/the-monument/history",
                snippet="The tower was built for the 1889 Exposition Universelle.",
            )
        ],
    ),
]

judge_payload = {
    "claim": claim,
    "plan": plan.model_dump(),
    "findings": [f.model_dump() for f in findings],
}

judge_payload

## 🚀 Run judge agent

In [ ]:
from agents import Runner
from facticli.agents import build_judge_agent
from facticli.types import FactCheckReport

model = os.getenv("FACTICLI_MODEL", "gpt-4.1-mini")
judge_agent = build_judge_agent(model=model)

result = await Runner.run(judge_agent, json.dumps(judge_payload, indent=2))
report = result.final_output_as(FactCheckReport, raise_if_incorrect_type=True)
report.model_dump()

## 🧪 Quick tweaks

- Flip one signal to `refutes` to test conflict handling.
- Lower confidences to test `Not Enough Evidence` behavior.
- Compare report changes after editing `src/facticli/prompts/judge.md`.